## Import library

In [34]:
import ee
import geemap as emap
import os
import ee.mapclient
import geetools as gee
import ipygee as ui

ModuleNotFoundError: No module named 'ipygee'

## Initialization 

In [2]:
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

### Display the Map viewer

In [3]:
Map=emap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

### Load country boundary feature- Singapore 

In [16]:
# Display the Hanoi boundary 
UN=ee.FeatureCollection("FAO/GAUL/2015/level2")

Singapore=UN.filterMetadata("ADM0_NAME","equals","Singapore")

Map.centerObject(Singapore,9)
Map.addLayer(ee.Image().paint(Singapore,0,1.5),{"palette":["black"]},"Singapore") # Colour = 0 or set color="red", and width =1.5

# Import Landsat 8 TOA Data

In [6]:
# Obtain Landsat 8 TOA Level 1
LS8=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2018-01-01","2018-12-31")\
.filterMetadata("CLOUD_COVER","less_than",10).filterBounds(Singapore) # Filter boundary of Singapore and cloud cover
Map.addLayer(LS8,{},"LS8")

In [66]:
# Show the number of images in the LS8 Collection (Singapore) with less than 10 cloud percent
print("The images cover Singapore is {} ".format(LS8.size().getInfo()))

The images cover Singapore is 2 


## Download a single image and image collection using ee package

## Download single image 

In [67]:
# Obtain median over the Image Collection
LS_Sing=LS8.median().float().clip(Singapore) # Convert to float32 as it has both float32 and float64. Can't download if mixed
Map.addLayer(LS_Sing,{},"LSSIng")

In [155]:
# Download a single image with boundary of Singapore and 30m pixel size
task = ee.batch.Export.image.toDrive(image=LS_Sing,  # an ee.Image object.
                                     region=Singapore.geometry().bounds().getInfo()["coordinates"],  # an ee.Geometry object.
                                     description="LS_Sing",
                                     folder="Examples",
                                     fileNamePrefix="LS_Sing",
                                     scale=30,
                                     crs="EPSG:4326")
task.start()

In [160]:
task.status()

{'state': 'COMPLETED',
 'description': 'LS_Sing',
 'creation_timestamp_ms': 1603006806278,
 'update_timestamp_ms': 1603006932809,
 'start_timestamp_ms': 1603006817208,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/12EBLRFpIRdapPpC2I2ErW-h1STdoEVEN'],
 'attempt': 1,
 'id': '5BUFTNTAIOHDZL5A4TFURSZS',
 'name': 'projects/earthengine-legacy/operations/5BUFTNTAIOHDZL5A4TFURSZS'}

## Download image collection for Singapore 

In [163]:
# Download all two images in the LS8 Collection (Singapore)
LS8=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2018-01-01","2018-12-31")\
.filterMetadata("CLOUD_COVER","less_than",10).filterBounds(Singapore) # Filter boundary of Singapore and cloud cover

In [32]:
# Loop over all images in the Collection and clip the study area and export

mlist =LS8.sort("system:time_start").aggregate_array("system:index").getInfo()
for eachimage in mlist:
    img=ee.Image("LANDSAT/LC08/C01/T1_TOA/{}".format(eachimage)).clip(Singapore)
    task = ee.batch.Export.image.toDrive(image=img,  # an ee.Image object
                                         region=Singapore.geometry().bounds().getInfo()["coordinates"],  # an ee.Geometry object.
                                         description=eachimage,
                                         folder="Examples",
                                         fileNamePrefix=eachimage,
                                         scale=30,
                                         crs="EPSG:4326")

    task.start() 

In [143]:
# We can pass namePattern by our defined pattern
task=gee.batch.Export.imagecollection.toDrive(
    LS8, 
    'Collection',
    namePattern='LS8_{system_date}_{CLOUD_COVER}', # This can be passed namePattern='LS8_{system_date}_{CLOUD_COVER}'
    scale=30,
    dataType="float", 
    region=Singapore.geometry().bounds().getInfo()["coordinates"], 
    datePattern="y-MM-dd", # Can be passed "y-MM-dd"
    extra=None, 
    verbose=False,
    crs="EPSG:4326"
)
task

[<Task EXPORT_IMAGE: LS8_2018-05-08_6.84 (UNSUBMITTED)>,
 <Task EXPORT_IMAGE: LS8_2018-05-24_5.67 (UNSUBMITTED)>]

# Some learnt notes

ee.Feature = Singapore.geometry() behaves like ee.geometry() # Here we can use all methods of ee.Feature

In [105]:
a=Singapore.geometry().coordinates().getInfo()

In [106]:
a[0][0][0]

[103.81230037896471, 1.3840826903216656]

In [107]:
Singapore.geometry().bounds().getInfo()["coordinates"]

[[[103.63828400206005, 1.1640393937452072],
  [104.09003995172597, 1.1640393937452072],
  [104.09003995172597, 1.4712714973692553],
  [103.63828400206005, 1.4712714973692553],
  [103.63828400206005, 1.1640393937452072]]]

In [26]:
list(os.path.splitext("SHape.file.shp"))

['SHape.file', '.shp']

In [25]:
a="Shape.file.shp"
a.split(".")

['Shape', 'file', 'shp']

In [27]:
a=os.path.splitext("SHape.file.shp")
a

('SHape.file', '.shp')

In [29]:
a[1].strip()

'.shp'